# Evaluation of different crowd analysis metrics


(1) Dataset filtering:

* Exclude workers based on:
    - contradiction ration (absolut thresh, batch stdv, pair stdv, total stdv)
    - worker quality score (thresholds)
    - attention check fails 
    
(2) Label aggregation:

* Majority vote
* Top vote
* CT unit-label score (tresholds)

In [1]:
# Test crowd truth aggregation here 

from clean_annotations import clean_workers
from load_data import load_experiment_data, load_expert_data
from aggregation import aggregate_binary_labels

# load expert data    
#run = '4'
#group = 'expert_inspection1'
#n_q = '*'
#batch = '*'
#data_dict_list_exp = load_expert_data(run, group, n_q, batch)

run = '4'
group = 'experiment2'
n_q = '*'
batch = '161'
metric = 'contradictions'
unit = 'batch'
n_stds = 1
data_dict_list_crowd = load_experiment_data(run, group, n_q, batch)
#data_dict_list_crowd_clean = clean_workers(data_dict_list_crowd, run,  group, 
 #                                          batch, metric, unit, n_stds)
#crowd_bin_labels_clean  = aggregate_binary_labels(data_dict_list_crowd)

Discarded 0.0 annotations.


In [2]:
# Test load expert data

# load expert data    
run = '4'
group = 'expert_inspection1'
n_q = '30'
batch = '1'
data_dict_list_exp = load_expert_data(run, group, n_q, batch)


odict_keys(['relation', 'property', 'concept', 'answer', 'agreement', 'workerid', 'questionid', 'quid', 'description', 'exampletrue', 'examplefalse', 'run', 'sublist', 'completionurl', 'name'])
odict_keys(['relation', 'property', 'concept', 'answer', 'agreement', 'workerid', 'questionid', 'quid', 'description', 'exampletrue', 'examplefalse', 'run', 'sublist', 'completionurl', 'name'])
odict_keys(['relation', 'property', 'concept', 'answer', 'agreement', 'workerid', 'questionid', 'quid', 'description', 'exampletrue', 'examplefalse', 'run', 'sublist', 'completionurl', 'name'])
odict_keys(['relation', 'property', 'concept', 'answer', 'agreement', 'workerid', 'questionid', 'quid', 'description', 'exampletrue', 'examplefalse', 'run', 'sublist', 'completionurl', 'name'])
odict_keys(['relation', 'property', 'concept', 'answer', 'agreement', 'workerid', 'questionid', 'quid', 'description', 'exampletrue', 'examplefalse', 'run', 'sublist', 'completionurl', 'name'])
odict_keys(['relation', 'prope

# Aggregate labels and evaluate against gold

In [1]:
import pandas as pd

from evaluation import evaluate 
from aggregation import aggregate_binary_labels, get_expert_labels
from load_data import load_expert_data
from load_data import load_experiment_data
from clean_annotations import clean_workers
from calculate_iaa import get_agreement 

def get_performance_overview(data_dict_list_exp, run, group, n_q, batch):
    data_dict_list_crowd = load_experiment_data(run, group, n_q, batch)
    units = [ 'raw', 'pair', 'batch', 'total']
    stds = ['raw', 0.5, 1, 1.5, 2]
    metrics = ['contradictions', 'crowdtruth']
    votes = ['majority_vote', 'top_vote', 'ct_vote_0.5', 'ct_vote_0.6',
             'ct_vote_0.7', 'ct_vote_0.8', 'ct_vote_0.9', 
             'top_vote_ct_0.5', 'top_vote_ct_0.6','top_vote_ct_0.7',
             'top_vote_ct_0.8', 'top_vote_ct_0.9']
    overview_dicts = []
    expert_bin_labels = get_expert_labels(data_dict_list_exp)
    
    for v in votes:
        print(v)
        if v.startswith('ct_vote') or v.startswith('top_vote_ct'):
            unit = 'raw'
            n_stdv = 'raw'
            data_dict_list_crowd_clean = data_dict_list_crowd
            crowd_bin_labels_clean  = aggregate_binary_labels(data_dict_list_crowd_clean)
            iaa = get_agreement(data_dict_list_crowd_clean, 
                                collapse_relations = False, v=False, disable_kappa = True)
            iaa_levels = get_agreement(data_dict_list_crowd_clean,
                                       collapse_relations = 'levels', v=False,
                                       disable_kappa = True)
            results_dict_rel = evaluate(expert_bin_labels, crowd_bin_labels_clean,\
                                        vote=v, label = 'relation')
            results_dict_level = evaluate(expert_bin_labels, crowd_bin_labels_clean,\
                                        vote=v, label = 'level')
            d = dict()
            d['cleaning'] = f'{unit}_{n_stdv}'
            d['aggregation'] = v
            d['f1_relation'] = results_dict_rel['f1']
            d['p_relation'] = results_dict_rel['p']
            d['r_relation'] = results_dict_rel['r']
            d['f1_level'] = results_dict_level['f1']
            d['iaa'] = iaa['Krippendorff']
            d['iaa_levels'] = iaa_levels['Krippendorff']
            overview_dicts.append(d)
        else:
            for unit in units: 
                for n_stdv in stds:
                    if unit == str(n_stdv) == 'raw':
                        data_dict_list_crowd_clean = data_dict_list_crowd
                        crowd_bin_labels_clean  = aggregate_binary_labels(data_dict_list_crowd_clean)
                    elif 'raw' not in [unit, n_stdv]:
                        for metric in metrics:
                            print(v, metric, unit, n_stdv)
                            data_dict_list_crowd_clean = clean_workers(data_dict_list_crowd, run, group, 
                                               batch, metric, unit, n_stdv)
                            crowd_bin_labels_clean  = aggregate_binary_labels(
                                                        data_dict_list_crowd_clean)
                            iaa = get_agreement(data_dict_list_crowd_clean, 
                                            collapse_relations = False, v=False, disable_kappa = True)
                            iaa_levels = get_agreement(data_dict_list_crowd_clean,
                                                       collapse_relations = 'levels', v=False, disable_kappa = True)

                            print(v)
                            results_dict_rel = evaluate(expert_bin_labels, crowd_bin_labels_clean,\
                                                    vote=v, label = 'relation')
                            results_dict_level = evaluate(expert_bin_labels, crowd_bin_labels_clean,\
                                                    vote=v, label = 'level')
                            # missing: write function to get iaa of selected labels only 
                            d = dict()
                            d['cleaning'] = f'{metric}_{unit}_{n_stdv}'
                            d['aggregation'] = v
                            d['f1_relation'] = results_dict_rel['f1']
                            d['p_relation'] = results_dict_rel['p']
                            d['r_relation'] = results_dict_rel['r']
                            d['f1_level'] = results_dict_level['f1']
                            d['iaa'] = iaa['Krippendorff']
                            d['iaa_levels'] = iaa_levels['Krippendorff']
                            overview_dicts.append(d)
                        else:
                            continue                  
    return overview_dicts

# load expert data    
run = '4'
group = 'expert_inspection1'
n_q = '30'
batch = '1'
data_dict_list_exp = load_expert_data(run, group, n_q, batch)

run = '*'
group = 'experiment*'
n_q = '*'
batch = '*'

# evaluate:
overview_dicts = get_performance_overview(data_dict_list_exp, run, group, n_q, batch)
df = pd.DataFrame(overview_dicts)

Discarded 655.0 annotations.
majority_vote
majority_vote contradictions pair 0.5
majority_vote
majority_vote crowdtruth pair 0.5
majority_vote
majority_vote contradictions pair 1
majority_vote
majority_vote crowdtruth pair 1
majority_vote
majority_vote contradictions pair 1.5
majority_vote
majority_vote crowdtruth pair 1.5
majority_vote
majority_vote contradictions pair 2
majority_vote
majority_vote crowdtruth pair 2
majority_vote
majority_vote contradictions batch 0.5
majority_vote
majority_vote crowdtruth batch 0.5
majority_vote
majority_vote contradictions batch 1
majority_vote
majority_vote crowdtruth batch 1
majority_vote
majority_vote contradictions batch 1.5
majority_vote
majority_vote crowdtruth batch 1.5
majority_vote
majority_vote contradictions batch 2
majority_vote
majority_vote crowdtruth batch 2
majority_vote
majority_vote contradictions total 0.5
majority_vote
majority_vote crowdtruth total 0.5
majority_vote
majority_vote contradictions total 1
majority_vote
majority_vot

In [2]:
df = df.sort_values('f1_relation', ascending = False).round(2)
df

,aggregation,cleaning,f1_level,f1_relation,iaa,iaa_levels,p_relation,r_relation
20,majority_vote,contradictions_total_1.5,1.00,0.97,0.34,0.54,0.97,0.97
8,majority_vote,contradictions_batch_0.5,1.00,0.97,0.37,0.60,0.97,0.97
18,majority_vote,contradictions_total_1,1.00,0.97,0.35,0.55,0.97,0.97
16,majority_vote,contradictions_total_0.5,1.00,0.97,0.36,0.58,0.97,0.97
2,majority_vote,contradictions_pair_1,0.92,0.94,0.37,0.63,0.95,0.93
0,majority_vote,contradictions_pair_0.5,0.92,0.90,0.38,0.64,0.91,0.90
49,ct_vote_0.6,raw_raw,1.00,0.90,0.33,0.52,0.91,0.90
40,top_vote,contradictions_total_0.5,0.92,0.90,0.36,0.58,0.90,0.90
13,majority_vote,crowdtruth_batch_1.5,0.92,0.87,0.33,0.52,0.91,0.87
23,majority_vote,crowdtruth_total_2,0.92,0.87,0.33,0.52,0.91,0.87


In [6]:
print(df.to_latex(index=False))

\begin{tabular}{llrrrrrr}
\toprule
     aggregation &             cleaning &  f1\_level &  f1\_relation &   iaa &  iaa\_levels &  p\_relation &  r\_relation \\
\midrule
   majority\_vote &  cont\_rate\_total\_1.5 &      1.00 &         0.97 &  0.33 &        0.52 &        0.97 &        0.97 \\
   majority\_vote &  cont\_rate\_total\_0.5 &      1.00 &         0.97 &  0.35 &        0.56 &        0.97 &        0.97 \\
   majority\_vote &  cont\_rate\_batch\_0.5 &      1.00 &         0.97 &  0.36 &        0.60 &        0.97 &        0.97 \\
   majority\_vote &    cont\_rate\_total\_1 &      1.00 &         0.97 &  0.34 &        0.54 &        0.97 &        0.97 \\
   majority\_vote &     cont\_rate\_pair\_1 &      0.92 &         0.94 &  0.36 &        0.60 &        0.95 &        0.93 \\
   majority\_vote &   cont\_rate\_pair\_0.5 &      0.92 &         0.90 &  0.37 &        0.60 &        0.91 &        0.90 \\
     ct\_vote\_0.6 &              raw\_raw &      1.00 &         0.90 &  0.32 &        

# Evaluate individual workers